# 完成版（未完成）　できるだけ機能を関数化するパターン

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import datetime
import time
import re # 正規表現用
import sys
import pandas as pd

# このプログラムで取得する Amazon・ヤフオク情報を格納するためのリスト準備
data_list = []

#ブラウザの設定
chrome_options = webdriver.ChromeOptions()
# options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
# 拡張機能 Resale Trap を有効にする　staff2@polyphony.tokyo／Muzaiko20230606
chrome_options.add_extension(r'C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\extensions/resaletrap_3.0.9.crx')
# Chromeを起動するためのオプションにプロファイルを追加する
chrome_options.add_argument(r'--user-data-dir=C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\profile')

#ブラウザの起動する
browser = webdriver.Chrome(r'C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\chromedriver.exe', options=chrome_options)
browser.implicitly_wait(3)

# Amazon商品ページを開く
# 固定入力パターン
url = 'https://www.amazon.co.jp/s?i=kitchen&bbn=3901642051&rh=n%3A3901642051%2Cn%3A3895771%2Cn%3A4083271&dc&fs=true&ds=v1%3AjNxS2PZfTpTxxP1weAHzLyoVu6VNah9Uop7hosBt8fU&qid=1686742341&rnid=3895771&ref=sr_nr_n_1'
# 手動入力パターン
# print('▼ 調査対象の Amazon URL を入力してください')
# url = input()
# print('')
browser.get(url)
browser.implicitly_wait(5)

# Amazon のウィンドウハンドラの保持
amazon_window = browser.current_window_handle

# 開いた Amazon ページからヤフオク商品ページを開く関数
def open_yafuoku_page():
    # Amazon商品＋ヤフオク商品群の基点要素を取得する
    amazon_yafuoku_item_list = browser.find_elements(By.XPATH, '//div[@data-component-type="s-search-result"]')

    # Amazon＋ヤフオク商品群の親要素内の情報を取得する
    for amazon_elem in amazon_yafuoku_item_list:

        # Amazon商品名
        amazon_name = amazon_elem.find_element(By.XPATH, './/h2//span').text
        # print('Amazon商品名：', amazon_name)

        # Amazon購入実績
        try:
            amazon_sold_num = amazon_elem.find_element(By.XPATH, './/span[contains(text(), "購入されました")]').text
        except:
            amazon_sold_num = 'Amazon購入実績情報なし'
        # finally:
            # print('Amazon購入実績：', amazon_sold_num)
    
        # Amazon価格
        try:
            amazon_price = int(amazon_elem.find_element(By.XPATH, './/span[@class="a-price-whole"]').text.replace(',', ''))
        except:
            amazon_price = 'Amazon価格情報なし'
        # finally:
            # print('Amazon価格：', amazon_price, '円')

        # Amazon商品URL
        amazon_url = amazon_elem.find_element(By.XPATH, './/h2/a').get_attribute('href')
        # print('Amazon商品URL：', amazon_url)

        # ヤフオク商品群の基点要素を取得する
        yafuoku_item_list = amazon_elem.find_elements(By.XPATH, './/tbody')

        # ヤフオク商品群の親要素内の情報を取得する
        for yafuoku_elem in yafuoku_item_list:

            print('Amazon商品名：', amazon_name)
            print('Amazon価格：', amazon_price)

            # ヤフオク商品名
            try:
                yafuoku_name = yafuoku_elem.find_element(By.XPATH, './/a[@class="__ylink"]').text
                print('ヤフオク商品名：', yafuoku_name)
            except:
                print('▲ 該当ヤフオク商品なし ▲')
                # pass

            # ヤフオク現在価格
            try:
                yafuoku_price_now = int(yafuoku_elem.find_element(By.XPATH, './/div[contains(text(), "現在")]').text.replace('現在 : ￥', ''))
                print('ヤフオク現在価格：', yafuoku_price_now)
            except:
                print('▲ ヤフオク現在価格なし ▲')
                # pass

            # 現在価格での利益額計算
            profit_now = amazon_price - yafuoku_price_now
            print('A-Y現在価格差(送料考慮前)：', profit_now)

            # 原罪価格での利益率計算
            profit_rate_now = profit_now / yafuoku_price_now
            print('A-Y現在利益率(送料考慮前)：', round(profit_rate_now*100, 1), '%')

            # 現在価格での利益額/率が基準値以上ならヤフオク商品ページを開いて処理を続ける
            if profit_rate_now > 0.1: # 利益率での判定

                # ヤフオク商品ページを開く
                yafuoku_elem.find_element(By.XPATH, './/a').send_keys(Keys.ENTER)

                print('---------------------------------------------------------------------------------')
                print('開いたヤフオク商品への対応を選んでください( YES / NO / END)')
                print('　YES：商品情報を取得する')
                print('　NO ：情報取得をスキップして次の商品に移る(商品違い/匿名配送/評価不足などの場合)')
                print('　END：プログラムを終了する')
                print('---------------------------------------------------------------------------------')
                
                while True:
                    ans = input()

                    if ans == 'YES':
                        # print('この商品の情報を取得します')
                        # ヤフオク商品ページの情報取得に移る ■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■
                        get_yafuoku_data()
                        break
                    elif ans == 'NO':
                        print('この商品の情報取得はスキップしました')
                        # 開いたヤフオク商品ページをアクティブにしてから閉じて Amazon ページをアクティブにする
                        handle_array = browser.window_handles
                        browser.switch_to.window(handle_array[-1])
                        browser.close()
                        browser.switch_to.window(amazon_window)
                        break
                    elif ans == 'END':
                        print('プログラムを終了します')
                        # ヤフオク商品ページと Amazon ページを閉じてプログラムを終了する
                        handle_array = browser.window_handles
                        browser.switch_to.window(handle_array[-1])
                        browser.close()
                        browser.switch_to.window(amazon_window)
                        browser.close()
                        sys.exit()
                    else:
                        print('YES か NO か END を入力してください')
            else:
                print('◆ 利益率が基準値未満のため情報取得をスキップしました ◆')
            
            print('')
        print('')
        
# ヤフオク商品ページから各種情報を取得する関数
def get_yafuoku_data():
    # 新たに開いたヤフオク商品ページをアクティブにする
    handle_array = browser.window_handles
    browser.switch_to.window(handle_array[-1])
    yafuoku_window = browser.current_window_handle
    # print('最新のヤフオク商品ページをアクティブにしました')

    # ヤフオク商品名を取得する
    yafuoku_name = browser.find_element(By.XPATH, '//h1').text
    print('ヤフオク商品名：', yafuoku_name)

    # 出品者の評価
    # 97％以下の場合は処理をスキップする
    seller_rating = browser.find_element(By.XPATH, '//div[@class="Seller__ratingRatio"]').text
    print('評価：', seller_rating)

    seller_rating_int = int(seller_rating.split('.')[0].replace('%', ''))
    
    if seller_rating_int < 97:
        print('◆ 評価が低いため本商品の情報収集を終了します ◆')
        browser.close()
        browser.switch_to.window(amazon_window)
        return

    # 評価数は多い方が望ましい
    seller_rating_num = browser.find_element(By.XPATH, '//a[@class="Seller__ratingLink"]').text
    print('評価数：', seller_rating_num)

    # 匿名配送情報
    # 匿名配送希望の場合は処理をスキップする
    try:
        # 匿名配送だったらその時点で情報取得をやめて画面を閉じる
        privacy = browser.find_element(By.XPATH, '//span[@class="Icon Icon--privacy"]').text # 匿名配送の場合
        print('◆ 匿名配送のため本商品の情報収集を終了します ◆')
        browser.close()
        browser.switch_to.window(amazon_window)
        return
    except:
        privacy = '匿名配送なし'
        print('匿名配送：', privacy)

    # ヤフオク商品ページ（アクティブウィンドウ）の URL を取得する
    yafuoku_url = browser.current_url
    print('ヤフオク商品URL：', yafuoku_url)
        
    # ヤフオク現在価格を取得する
    yafuoku_price_now = browser.find_element(By.XPATH, '//dd[@class="Price__value"]').text.replace(' ', '').replace(',', '')
    print('ヤフオク現在価格：', yafuoku_price_now)

    # 後の利益計算のために int 型にしておく
    if '税込' in yafuoku_price_now:
        yafuoku_price_now_int = int(yafuoku_price_now.replace('税込', '／').replace('円）', '').split('／')[1])
    else:
        yafuoku_price_now_int = int(yafuoku_price_now.split('円')[0])
        
    # 商品の状態
    state = browser.find_element(By.XPATH, '//th[contains(text(), "状態")]/following-sibling::td/a').text
    print('状態：', state)
    
    # 入札数
    bidnum = browser.find_elements(By.XPATH, '//span[@class="Count__detail"]')[0].text
    print('入札数：', bidnum)

    browser.close()
    browser.switch_to.window(amazon_window)

    # 送料
    # ★
    
    # 商品説明
    # ★
    
    # 直接引き取り判定
    # ★

    # 着払い判定
    # ★

    # 配送業者、営業所止め判定
    # ★

    # 訳あり品判定
    # ★

    # オークション残り日数
    # ★

    # ヤフオク送料も加味した、現在価格での利益額/利益率判定　※ Amazon 出品送料は ResaleTrap で判定
    # ★

# 取得した情報を CSV 出力用にまとめる（出品判定の証跡として残す）関数
def store_data():
    data = {
        'Amazon商品名': amazon_name,
        'Amazon価格': amazon_price,
        'Amazon商品URL': amazon_url,
        'ヤフオク商品名': yafuoku_name,
        'ヤフオク現在価格': yafuoku_price_now,
        'ヤフオク商品URL': yafuoku_url,
        'A-Y現在利益(送料込)': profit,
        '現在利益率(送料込)': profit_rate,
        '情報取得日時': datetime.datetime.now().strftime('%Y/%m/%d %H:%M'),
        }

    data_list.append(data)
    print('data_list', data_list)

# データ格納用リストに取得した情報を CSV ファイルに出力する関数
def output():
    output_time = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
    csv_file_name = 'ヤフオクtoアマゾンデータ_' + output_time + '.csv'
    pd.DataFrame(data_list).to_csv(csv_file_name, encoding='cp932')
        
def main():
    open_yafuoku_page()
    # 取得したデータをリストに格納する
    # ■ どうやる？？？
    # store_data()
    # output()

if __name__ == '__main__':
    main()

# 完成版（未完成）　モノリシック化パターン

In [58]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import datetime
import time
import re # 正規表現用
import sys
import pandas as pd

# このプログラムで取得する Amazon・ヤフオク情報を格納するためのリスト準備
data_list = []

#ブラウザの設定
chrome_options = webdriver.ChromeOptions()
# options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
# 拡張機能 Resale Trap を有効にする　staff2@polyphony.tokyo／Muzaiko20230606
chrome_options.add_extension(r'C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\extensions/resaletrap_3.0.9.crx')
# Chromeを起動するためのオプションにプロファイルを追加する
chrome_options.add_argument(r'--user-data-dir=C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\profile')

#ブラウザの起動する
browser = webdriver.Chrome(r'C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\chromedriver.exe', options=chrome_options)
browser.implicitly_wait(3)

# Amazon商品ページを開く
# 固定入力パターン
url = 'https://www.amazon.co.jp/s?i=kitchen&bbn=3901642051&rh=n%3A3901642051%2Cn%3A3895771%2Cn%3A4083271&dc&fs=true&ds=v1%3AjNxS2PZfTpTxxP1weAHzLyoVu6VNah9Uop7hosBt8fU&qid=1686742341&rnid=3895771&ref=sr_nr_n_1'
# 手動入力パターン
# print('▼ 調査対象の Amazon URL を入力してください')
# url = input()
# print('')
browser.get(url)
browser.implicitly_wait(5)

# Amazon のウィンドウハンドラの保持
amazon_window = browser.current_window_handle

def main():
    # Amazon商品＋ヤフオク商品群の基点要素を取得する
    amazon_yafuoku_item_list = browser.find_elements(By.XPATH, '//div[@data-component-type="s-search-result"]')

    # Amazon＋ヤフオク商品群の親要素内の情報を取得する
    for amazon_elem in amazon_yafuoku_item_list:
       
        # Amazon商品名
        amazon_name = amazon_elem.find_element(By.XPATH, './/h2//span').text
        # print('Amazon商品名：', amazon_name)

        # Amazon購入実績
        try:
            amazon_sold_num = amazon_elem.find_element(By.XPATH, './/span[contains(text(), "購入されました")]').text
        except:
            amazon_sold_num = 'Amazon購入実績情報なし'
        # finally:
            # print('Amazon購入実績：', amazon_sold_num)
            
        # Amazon価格
        try:
            amazon_price = int(amazon_elem.find_element(By.XPATH, './/span[@class="a-price-whole"]').text.replace(',', ''))
        except:
            amazon_price = 'Amazon価格情報なし'
        # finally:
            # print('Amazon価格：', amazon_price)

        # Amazon商品URL
        amazon_url = amazon_elem.find_element(By.XPATH, './/h2/a').get_attribute('href')
        # print('Amazon商品URL：', amazon_url)

        # ヤフオク商品群の基点要素を取得する
        yafuoku_item_list = amazon_elem.find_elements(By.XPATH, './/tbody')

        # ヤフオク商品群の親要素内の情報を取得する
        for yafuoku_elem in yafuoku_item_list:

            print('Amazon商品名：', amazon_name)
            print('Amazon価格：', amazon_price)

            # ヤフオク商品名
            try:
                yafuoku_name = yafuoku_elem.find_element(By.XPATH, './/a[@class="__ylink"]').text
                print('ヤフオク商品名：', yafuoku_name)
            except:
                print('▲ 該当ヤフオク商品なし ▲')
                # pass

            # ヤフオク現在価格
            try:
                yafuoku_price_now = int(yafuoku_elem.find_element(By.XPATH, './/div[contains(text(), "現在")]').text.replace('現在 : ￥', ''))
                print('ヤフオク現在価格：', yafuoku_price_now)
            except:
                print('▲ ヤフオク現在価格なし ▲')
                # pass

            # 現在価格での利益額計算
            profit_now = amazon_price - yafuoku_price_now
            print('A-Y現在価格差(送料考慮前)：', profit_now)

            # 原罪価格での利益率計算
            profit_rate_now = profit_now / yafuoku_price_now
            print('A-Y現在利益率(送料考慮前)：', round(profit_rate_now*100), '%')

            # 現在価格での利益額/率が基準値以上ならヤフオク商品ページを開いて処理を続ける
            if profit_rate_now > 0.1: # 利益率での判定

                # ヤフオク商品ページを開く
                yafuoku_elem.find_element(By.XPATH, './/a').send_keys(Keys.ENTER)

                print('---------------------------------------------------------------------------------')
                print('開いたヤフオク商品への対応を選んでください( yes / no / end)')
                print('　yes：商品情報を取得する')
                print('　no ：情報取得をスキップして次の商品に移る(商品違い/匿名配送/評価不足などの場合)')
                print('　end：プログラムを終了する')
                print('---------------------------------------------------------------------------------')
                
                while True:
                    ans = input()

                    if ans == 'yes':
                        # print('この商品の情報を取得します')
                        # 新たに開いたヤフオク商品ページをアクティブにする
                        handle_array = browser.window_handles
                        browser.switch_to.window(handle_array[-1])
                        yafuoku_window = browser.current_window_handle
                        # print('最新のヤフオク商品ページをアクティブにしました')

                        # ヤフオク商品名を取得する
                        yafuoku_name = browser.find_element(By.XPATH, '//h1').text
                        print('ヤフオク商品名：', yafuoku_name)

                        # 出品者の評価
                        # 97％以下の場合は処理をスキップする
                        seller_rating = browser.find_element(By.XPATH, '//div[@class="Seller__ratingRatio"]').text
                        print('評価：', seller_rating)

                        seller_rating_int = int(seller_rating.split('.')[0].replace('%', ''))

                        if seller_rating_int < 97:
                            print('◆ 評価が低いため本商品の情報収集を終了します ◆')
                            browser.close()
                            browser.switch_to.window(amazon_window)
                            break

                        # 評価数は多い方が望ましい
                        seller_rating_num = browser.find_element(By.XPATH, '//a[@class="Seller__ratingLink"]').text
                        print('評価数：', seller_rating_num)

                        # 匿名配送情報
                        # 匿名配送希望の場合は処理をスキップする
                        try:
                            # 匿名配送だったらその時点で情報取得をやめて画面を閉じる
                            privacy = browser.find_element(By.XPATH, '//span[@class="Icon Icon--privacy"]').text # 匿名配送の場合
                            print('◆ 匿名配送のため本商品の情報収集を終了します ◆')
                            browser.close()
                            browser.switch_to.window(amazon_window)
                            break
                        except:
                            privacy = '匿名配送なし'
                            print('匿名配送：', privacy)

                        # ヤフオク商品ページ（アクティブウィンドウ）の URL を取得する
                        yafuoku_url = browser.current_url
                        print('ヤフオク商品URL：', yafuoku_url)

                        # ヤフオク現在価格を取得する
                        yafuoku_price_now = browser.find_element(By.XPATH, '//dd[@class="Price__value"]').text.replace(' ', '').replace(',', '')
                        print('ヤフオク現在価格：', yafuoku_price_now)

                        # 後の利益計算のために int 型にしておく
                        if '税込' in yafuoku_price_now:
                            yafuoku_price_now_int = int(yafuoku_price_now.replace('税込', '／').replace('円）', '').split('／')[1])
                        else:
                            yafuoku_price_now_int = int(yafuoku_price_now.split('円')[0])

                        # 商品の状態
                        state = browser.find_element(By.XPATH, '//th[contains(text(), "状態")]/following-sibling::td/a').text
                        print('状態：', state)

                        # 入札数
                        bidnum = browser.find_elements(By.XPATH, '//span[@class="Count__detail"]')[0].text
                        print('入札数：', bidnum)

                        # 送料
                        # 送料の「詳細」ポップアップを開く
                        browser.find_element(By.XPATH, '//a[@id="postageDetailCurrent"]').click()
                        browser.implicitly_wait(1)
                        # 配送先から「青森県」を選ぶ
                        browser.find_element(By.XPATH, '//option[contains(text(), "青森県")]').click()
                        browser.implicitly_wait(1)

                        postage_due = browser.find_element(By.XPATH, '//dt[contains(text(), "送料負担")]/following-sibling::dd').text
                        print('送料負担：', postage_due)

                        # 配送情報を取得する
                        try:
                            shipping = browser.find_element(By.XPATH, '//dt[@class="BidModal__postageName"]').text
                            print('配送方法(詳細)：', shipping)

                            postage = browser.find_element(By.XPATH, '//div[@class="BidModal__postagePrice"]').text
                            print('送料(詳細)：', postage)
                            
                            if postage == '無料':
                                postage_int = 0
                            else:
                                postage_int = int(postage.replace('円（税込）', ''))
                            print('送料(int)：', postage_int)
                        except:
                            pass

                        # 「✕」をクリックしてポップアップを閉じる
                        browser.find_element(By.XPATH, '//div[@class="BidModal__box BidModal__box--postage js-modal-box"]//a[contains(text(), "閉じる")]').click()

                        postage_from = browser.find_element(By.XPATH, '//p[@class="Price__postageFrom"]').text
                        print(postage_from)

                        try:
                            send_date = browser.find_element(By.XPATH, '//dt[contains(text(), "発送開始")]/following-sibling::dd').text
                        except:
                            send_date = '情報なし'
                        finally:
                            print('発送開始：', send_date)

                        try:
                            postage_2 = browser.find_element(By.XPATH, '//span[@class="Price__postageValue"]').text
                        except:
                            postage_2 = '情報なし'
                        finally:
                            print('送料(地域込)：', postage_2)

                        # 商品説明
                        yafuoku_info = browser.find_element(By.XPATH, '//div[@class="ProductExplanation__commentArea"]').text
                        # print('商品説明：', yafuoku_info)

                        # 直接引き取り判定
                        print('直接引取(商品名)：', re.findall('.*直接.*引.*', yafuoku_name))
                        print('直接引取(商品説明)：', re.findall('.*直接.*引.*', yafuoku_info))
                        print('手渡し(商品説明)', re.findall('.*手渡.*', yafuoku_info))

                        # 着払い判定
                        print('送料負担：', postage_due) # 上で取得済
                        print('着払(商品名)：', re.findall('.*着払.*', yafuoku_name))
                        print('着払(商品説明)：', re.findall('.*着払.*', yafuoku_info))

                        # 配送業者、営業所止め判定
                        print('ヤマト(商品名)：', re.findall('.*ヤマト.*', yafuoku_name))
                        print('ヤマト(商品説明)：', re.findall('.*ヤマト.*', yafuoku_info))
                        print('らくらく家財便(商品説明)：', re.findall('.*らくらく.*', yafuoku_info))
                        print('佐川(商品説明)', re.findall('.*佐川.*', yafuoku_info))
                        print('日本郵便(商品説明)', re.findall('.*日本郵便.*', yafuoku_info))
                        print('日本郵便(商品説明)', re.findall('.*ゆう.*', yafuoku_info))
                        print('営業所止め(商品説明)', re.findall('.*止め.*', yafuoku_info))

                        # 訳あり品判定
                        print('訳あり(商品名)：', re.findall('.*訳.*', yafuoku_name))
                        print('訳あり(商品説明)：', re.findall('.*訳.*', yafuoku_info))

                        # オークション残り日数
                        auc_timeleft = browser.find_elements(By.XPATH, '//span[@class="Count__detail"]')[1].text
                        print('残り日数(画面表示)：', auc_timeleft)

                        auc_enddatetime = browser.find_element(By.XPATH, '//th[contains(text(), "終了日時")]/following-sibling::td').text
                        print('終了予定(str型)：', auc_enddatetime)

                        auc_enddate = auc_enddatetime.split('（')[0]
                        auc_endtime = auc_enddatetime.split('）')[1]
                        auc_end = auc_enddate + ' ' + auc_endtime
                        auc_enddatetime = datetime.datetime.strptime(auc_end, '%Y.%m.%d %H:%M')
                        print('終了予定(datetime型)：', auc_enddatetime)

                        dt_now = datetime.datetime.now()
                        print('現在時刻：', dt_now.strftime('%Y-%m-%d %H:%M:%S'))

                        auc_time_left = auc_enddatetime - dt_now
                        print('残り時間(詳細計算)：', auc_time_left)

                        # ヤフオク送料も加味した、現在価格での利益額/利益率判定　※ Amazon 出品送料は ResaleTrap で判定
                        profit_now_final = amazon_price - yafuoku_price_now_int - postage_int
                        print('A-Y現在利益(ヤフオク送料込)：', profit_now_final, '円　※Amazon出品手数料は含まれていません')

                        profit_rate_now_final = round(profit_now_final / yafuoku_price_now_int * 100, 1)
                        print('現在利益率(ヤフオク送料込)：', profit_rate_now_final, '%　※Amazon出品手数料は含まれていません')

                        # 取得したデータをリストに格納する
                        data = {
                            'Amazon商品名': amazon_name,
                            'Amazon価格': amazon_price,
                            'Amazon商品URL': amazon_url,
                            'ヤフオク商品名': yafuoku_name,
                            'ヤフオク現在価格': yafuoku_price_now_int,
                            'ヤフオク商品URL': yafuoku_url,
                            'A-Y現在利益(送料込)': profit_now_final,
                            '現在利益率(送料込)': round(profit_rate_now_final, 3),
                            '情報取得日時': datetime.datetime.now().strftime('%Y/%m/%d %H:%M'),
                            }
                        data_list.append(data)
                        
                        browser.close()
                        browser.switch_to.window(amazon_window)

                        break
                    elif ans == 'no':
                        print('この商品の情報取得はスキップしました')
                        # 開いたヤフオク商品ページをアクティブにしてから閉じて Amazon ページをアクティブにする
                        handle_array = browser.window_handles
                        browser.switch_to.window(handle_array[-1])
                        browser.close()
                        browser.switch_to.window(amazon_window)
                        break
                    elif ans == 'end':
                        # 取得した情報を CSV ファイルに出力する
                        print('取得情報をファイルに出力してプログラムを終了します')
                        output_time = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
                        csv_file_name = 'ヤフオクtoアマゾンデータ_' + output_time + '.csv'
                        pd.DataFrame(data_list).to_csv(csv_file_name, encoding='cp932')

                        # ヤフオク商品ページと Amazon ページを閉じてプログラムを終了する
                        handle_array = browser.window_handles
                        browser.switch_to.window(handle_array[-1])
                        browser.close()
                        browser.switch_to.window(amazon_window)
                        browser.close()
                        sys.exit()
                    else:
                        print('yes か no か end を入力してください')
            else:
                print('◆ 利益率が基準値未満のため情報取得をスキップしました ◆')
            
            print('')
        print('')

if __name__ == '__main__':
    main()

C:\Users\amuza\AppData\Local\Temp\ipykernel_8980\1741558654.py:25: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(r'C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\chromedriver.exe', options=chrome_options)


Amazon商品名： 【セット】パナソニック オーブンレンジ 26L フラットテーブル ブラック NE-MS4A-K + iwaki AGCテクノグラス 耐熱ガラス ボウル 丸型 1.5L 外径21.3cm BC323
Amazon価格： 29900
ヤフオク商品名： (中古品)iwaki(イワキ) ボウル 耐熱ガラス 丸型 1.5L 外径21.3cm KBC323
ヤフオク現在価格： 3480
A-Y現在価格差(送料考慮前)： 26420
A-Y現在利益率(送料考慮前)： 759 %
---------------------------------------------------------------------------------
開いたヤフオク商品への対応を選んでください( yes / no / end)
　yes：商品情報を取得する
　no ：情報取得をスキップして次の商品に移る(商品違い/匿名配送/評価不足などの場合)
　end：プログラムを終了する
---------------------------------------------------------------------------------


 no


この商品の情報取得はスキップしました

Amazon商品名： 【セット】パナソニック オーブンレンジ 26L フラットテーブル ブラック NE-MS4A-K + iwaki AGCテクノグラス 耐熱ガラス ボウル 丸型 1.5L 外径21.3cm BC323
Amazon価格： 29900
ヤフオク商品名： オーブン レンジ
ヤフオク現在価格： 6525
A-Y現在価格差(送料考慮前)： 23375
A-Y現在利益率(送料考慮前)： 358 %
---------------------------------------------------------------------------------
開いたヤフオク商品への対応を選んでください( yes / no / end)
　yes：商品情報を取得する
　no ：情報取得をスキップして次の商品に移る(商品違い/匿名配送/評価不足などの場合)
　end：プログラムを終了する
---------------------------------------------------------------------------------


 yes


...

 yes


ヤフオク商品名： Panasonic◆オーブンレンジ NE-T15A4-W
評価： 98.5%
評価数： 13,360
匿名配送： 匿名配送なし
ヤフオク商品URL： https://page.auctions.yahoo.co.jp/jp/auction/w1090032999
ヤフオク現在価格： 15900円（税込17490円）
状態： 目立った傷や汚れなし
入札数： 0件
送料負担： 落札者
配送方法(詳細)： ゆうパック
送料(詳細)： 770円（税込）
送料(int)： 770
発送元：埼玉県
発送開始： 情報なし
送料(地域込)： 青森県は770円（税込）（離島を除く）
直接引取(商品名)： []
直接引取(商品説明)： []
手渡し(商品説明) []
送料負担： 落札者
着払(商品名)： []
着払(商品説明)： []
ヤマト(商品名)： []
ヤマト(商品説明)： []
らくらく家財便(商品説明)： []
佐川(商品説明) []
日本郵便(商品説明) []
日本郵便(商品説明) []
営業所止め(商品説明) []
訳あり(商品名)： []
訳あり(商品説明)： []
残り日数(画面表示)： 4日
終了予定(str型)： 2023.06.20（火）13:25
終了予定(datetime型)： 2023-06-20 13:25:00
現在時刻： 2023-06-15 22:11:34
残り時間(詳細計算)： 4 days, 15:13:25.842010
A-Y現在利益(ヤフオク送料込)： 3987 円　※Amazon出品手数料は含まれていません
現在利益率(ヤフオク送料込)： 22.8 %　※Amazon出品手数料は含まれていません

Amazon商品名： パナソニック オーブンレンジ 電子レンジ 15L ターンテーブル 遠赤ヒーター 重量センサー ヘルツフリー ホワイト NE-T15A4-W
Amazon価格： 22247
ヤフオク商品名： Panasonic◆オーブンレンジ NE-T15A4-W
ヤフオク現在価格： 16390
A-Y現在価格差(送料考慮前)： 5857
A-Y現在利益率(送料考慮前)： 36 %
--------------------------------------------------------------------

 yes


ヤフオク商品名： Panasonic◆オーブンレンジ NE-T15A4-W
評価： 98.5%
評価数： 13,360
匿名配送： 匿名配送なし
ヤフオク商品URL： https://page.auctions.yahoo.co.jp/jp/auction/j1090039397
ヤフオク現在価格： 14900円（税込16390円）
状態： やや傷や汚れあり
入札数： 0件
送料負担： 落札者
配送方法(詳細)： ゆうパック
送料(詳細)： 770円（税込）
送料(int)： 770
発送元：東京都
発送開始： 情報なし
送料(地域込)： 青森県は770円（税込）（離島を除く）
直接引取(商品名)： []
直接引取(商品説明)： []
手渡し(商品説明) []
送料負担： 落札者
着払(商品名)： []
着払(商品説明)： []
ヤマト(商品名)： []
ヤマト(商品説明)： []
らくらく家財便(商品説明)： []
佐川(商品説明) []
日本郵便(商品説明) []
日本郵便(商品説明) []
営業所止め(商品説明) []
訳あり(商品名)： []
訳あり(商品説明)： []
残り日数(画面表示)： 4時間
終了予定(str型)： 2023.06.16（金）01:12
終了予定(datetime型)： 2023-06-16 01:12:00
現在時刻： 2023-06-15 22:11:40
残り時間(詳細計算)： 3:00:19.298005
A-Y現在利益(ヤフオク送料込)： 5087 円　※Amazon出品手数料は含まれていません
現在利益率(ヤフオク送料込)： 31.0 %　※Amazon出品手数料は含まれていません


Amazon商品名： アイリスオーヤマ 電子レンジ 18L 単機能レンジ 700W フラットテーブル 60Hz/西日本 ブラック PMB-F185-6
Amazon価格： Amazon価格情報なし
ヤフオク商品名： IRIS OHYAMA◆レンジ/PMB-F185-6-B
ヤフオク現在価格： 7590


TypeError: unsupported operand type(s) for -: 'str' and 'int'

# 関数間での変数の値を受け渡すテスト

In [ ]:
def outer():
    def inner():
        a = 2
        b = a + 1
        return b

    b = inner() # bという変数を作り、戻り値を渡す。

    c = str(b)
    print(c)

outer()

In [ ]:
def open_yafuoku_page_test():
    print('run open_yafuoku_page_test()')
    amazon = 'amazon_name'
    get_yafuoku_data_test(amazon)
    
def get_yafuoku_data_test(amazon):
    print('run get_yafuoku_data_test()')
    print('name:', amazon) # OK
    yafuoku = 'yafuoku_name'
    store_data_test(amazon, yafuoku)
    
def store_data_test(amazon, yafuoku):
    print('run store_data_test()')
    print(amazon)
    print(yafuoku)

def output_test():
    print('run output_test()')
    print(amazon)
    print(yafuoku)

def main_test():
    print('run main_test()')
    open_yafuoku_page_test()
    
main_test()
# store_data_test()
output_test()